In [102]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [287]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# IMPORTANTE TESTAR COM O ARQUIVO DE PREVISÃO GERADO NA DIFICULDADE DIFÍCIL, PARA CONSEGUIR TESTAR A MANIPULAÇÃO NO MODO FÁCIL E NORMAL
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

dificuldades = ['normal', 'facil', 'dificil']
criptos = ['Bitcoin', 'Ethereum', 'BNB', 'Solana']

acuracia = []

# Função para diminuir a acertividade do bot no modo facil e normal
def forcar_erros(df, taxa_erro_desejada):
    # Calcular número de previsões a serem alteradas
    n_modificar = int(len(df) * taxa_erro_desejada)
    
    # Selecionar índices aleatórios das previsões a modificar
    indices_modificar = np.random.choice(df.index, size=n_modificar, replace=False)
    
    # Inverter o sinal das previsões selecionadas
    df.loc[indices_modificar, 'Variação Prevista (%)'] *= -1
    
    return df

for dificuldade in dificuldades:
    for cripto in criptos:
        i=1
        while i<=30:
            
            base_diretório = Path(os.getcwd())
            diretório_princapl = base_diretório.parent
            caminho = diretório_princapl / 'data' / f'previsão semanal - {cripto}.csv'

            # Ler arquivo da base
            acao = pd.read_csv(caminho)
            
            df_semana = acao
            
            # Aplicar erro conforme a dificuldade
            if dificuldade == 'normal':
                df_semana = forcar_erros(df_semana, 0.50)  # Força erro em 50% das previsões
            elif dificuldade == 'facil':
                df_semana = forcar_erros(df_semana, 0.77)  # Força erro em 77% das previsões
            
            # Calcular se a direção do valor previsto foi igual ao real (1 = acertou | 0 = errou)
            df_semana['direcao_correta'] = np.where(
                (df_semana['Variação Real (%)'] > 0) & (df_semana['Variação Prevista (%)'] > 0) |
                (df_semana['Variação Real (%)'] < 0) & (df_semana['Variação Prevista (%)'] < 0),
                1, 0
            )

            # Calcular a métrica de Erro de Direção (Directional Accuracy)
            directional_accuracy = df_semana['direcao_correta'].mean()

            # Exibir resultados
            print(f"{i}- Directional Accuracy da {cripto} no modo {dificuldade}: {directional_accuracy * 100:.2f}%")
            # acuracia.append(directional_accuracy)
            # Armazenando os resultados
            acuracia.append({
                'criptomoeda': cripto,
                'dificuldade': dificuldade,
                'directional accuracy (%)': directional_accuracy*100
            })
            i+=1

results_df = pd.DataFrame(acuracia)


1- Directional Accuracy da Bitcoin no modo normal: 48.28%
2- Directional Accuracy da Bitcoin no modo normal: 52.87%
3- Directional Accuracy da Bitcoin no modo normal: 49.43%
4- Directional Accuracy da Bitcoin no modo normal: 50.57%
5- Directional Accuracy da Bitcoin no modo normal: 49.43%
6- Directional Accuracy da Bitcoin no modo normal: 47.13%
7- Directional Accuracy da Bitcoin no modo normal: 50.57%
8- Directional Accuracy da Bitcoin no modo normal: 51.72%
9- Directional Accuracy da Bitcoin no modo normal: 51.72%
10- Directional Accuracy da Bitcoin no modo normal: 43.68%
11- Directional Accuracy da Bitcoin no modo normal: 48.28%
12- Directional Accuracy da Bitcoin no modo normal: 44.83%
13- Directional Accuracy da Bitcoin no modo normal: 54.02%
14- Directional Accuracy da Bitcoin no modo normal: 45.98%
15- Directional Accuracy da Bitcoin no modo normal: 51.72%
16- Directional Accuracy da Bitcoin no modo normal: 51.72%
17- Directional Accuracy da Bitcoin no modo normal: 55.17%
18- Di

In [288]:
results_df

,criptomoeda,dificuldade,directional accuracy (%)
0,Bitcoin,normal,48.275862
1,Bitcoin,normal,52.873563
2,Bitcoin,normal,49.425287
3,Bitcoin,normal,50.574713
4,Bitcoin,normal,49.425287
...,...,...,...
355,Solana,dificil,85.057471
356,Solana,dificil,85.057471
357,Solana,dificil,85.057471
358,Solana,dificil,85.057471


In [289]:
# results_df.to_excel("Resultados testes dificuldades/Resultado geral dificuldade.xlsx", engine="openpyxl", index=False)

# Agrupar pelas combinações de parâmetros e calcular a média das métricas
resultados_agrupados = results_df.groupby(['criptomoeda', 'dificuldade']).agg({
    'directional accuracy (%)': 'mean'
}).reset_index()

# Exibir os resultados agrupados
print(resultados_agrupados)

# resultados_agrupados.to_excel("Resultados testes dificuldades/Média resultado geral dificuldade.xlsx", engine="openpyxl", index=False)

   criptomoeda dificuldade  directional accuracy (%)
0          BNB     dificil                 83.908046
1          BNB       facil                 32.988506
2          BNB      normal                 50.383142
3      Bitcoin     dificil                 86.206897
4      Bitcoin       facil                 29.808429
5      Bitcoin      normal                 49.846743
6     Ethereum     dificil                 86.206897
7     Ethereum       facil                 30.613027
8     Ethereum      normal                 49.808429
9       Solana     dificil                 85.057471
10      Solana       facil                 30.421456
11      Solana      normal                 50.613027
